## Pytorchのクイックスタート

各APIに関しては公式ドキュメントを参照することを推奨。

### ライブラリのimport

使うやつimportするだけ。
isortなどを用いて、見やすいようにimportする。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


### データセットの作成

自分が解きたい課題がある場合(応用)はカスタムのデータセットを用意する。
提案手法を確かめたり、簡単なデータを使ったりする場合はPyTorchが提供するデータセットなどを利用する。

In [2]:
training_data = datasets.FashionMNIST(
    root="/data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="/data", train=False, download=True, transform=ToTensor()
)


100%|██████████| 26421880/26421880 [00:11<00:00, 2228634.21it/s]


Extracting /data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 105084.04it/s]


Extracting /data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:07<00:00, 591187.24it/s]


Extracting /data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 12015735.67it/s]

Extracting /data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /data/FashionMNIST/raw



### データローダーの作成

基本的にデータセットをバッチに分けてDataLoaderというものを作成する。学習時に使用する。

In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break


Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


### モデルの作成

モデルはnn.Moduleを継承して作成する。構造と伝播方式を決める。

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 損失関数と最適化手法の作成

損失関数は基本的にnn.~Lossを組み合わせて作成。カスタムの損失関数を設計している人もいる。

最適化手法に関してはAdamを使いがち、スケジューリングなどもする。

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


### 学習時の関数を作成

関数化することを推奨。dataloader、model、loss function、optimizerを使用。

（型をつけても__call__()の引数と戻り値は補完されないっぽい？）

In [8]:
def train(
    dataloader: DataLoader,
    model: NeuralNetwork,
    loss_fn: nn.CrossEntropyLoss,
    optimizer: torch.optim.SGD,
) -> None:
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred: torch.Tensor = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, cur = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{cur:>5d} / {size:>5d}]")


### 推論時の関数を作成

関数化することを推奨。dataloader、model、loss functionを使用。

In [9]:
def test(
    dataloader: DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss
) -> None:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred: torch.Tensor = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )


### 学習と推論

In [10]:
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")


Epoch 1
-------------------------------
loss: 2.301820 [   64 / 60000]
loss: 2.290018 [ 6464 / 60000]
loss: 2.271774 [12864 / 60000]
loss: 2.272327 [19264 / 60000]
loss: 2.248667 [25664 / 60000]
loss: 2.232768 [32064 / 60000]
loss: 2.232696 [38464 / 60000]
loss: 2.200930 [44864 / 60000]
loss: 2.197880 [51264 / 60000]
loss: 2.182165 [57664 / 60000]
Test Error: 
 Accuracy: 54.5%, Avg loss: 2.165721 

Epoch 2
-------------------------------
loss: 2.170702 [   64 / 60000]
loss: 2.157637 [ 6464 / 60000]
loss: 2.107023 [12864 / 60000]
loss: 2.133658 [19264 / 60000]
loss: 2.067456 [25664 / 60000]
loss: 2.031221 [32064 / 60000]
loss: 2.052110 [38464 / 60000]
loss: 1.976033 [44864 / 60000]
loss: 1.983536 [51264 / 60000]
loss: 1.928017 [57664 / 60000]
Test Error: 
 Accuracy: 58.5%, Avg loss: 1.910562 

Epoch 3
-------------------------------
loss: 1.939190 [   64 / 60000]
loss: 1.903834 [ 6464 / 60000]
loss: 1.794050 [12864 / 60000]
loss: 1.845724 [19264 / 60000]
loss: 1.716707 [25664 / 60000]
l

### 学習結果の保存

保存し忘れたり、上書きしたりすると実験がパーになるので忘れずに。

In [14]:
torch.save(model.state_dict(), "/data/FashionMNIST/model.pth")
print("Saved PyTorch Model State to model.pth")


Saved PyTorch Model State to model.pth


### 保存した重みの読み込み

In [15]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("/data/FashionMNIST/model.pth"))


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 結果の詳細を確認

実際の正答を確認し、可視化などをする。

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred: torch.Tensor = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


tensor([-1.9879, -2.6412, -0.8158, -1.4558, -0.6634,  2.4251, -0.9630,  2.6783,
         1.8143,  3.0877], device='cuda:0')
Predicted: "Ankle boot", Actual: "Ankle boot"
